## Cleaning Data with PySpark

Working with data is tricky - working with millions or even billions of rows is worse. Did you receive some data processing code written on a laptop with fairly pristine data? Chances are you’ve probably been put in charge of moving a basic data process from prototype to production. You may have worked with real world datasets, with missing fields, bizarre formatting, and orders of magnitude more data. Even if this is all new to you, this notebook helps you learn what’s needed to prepare data processes using Python with Apache Spark. 

### Defining a schema

- Creating a defined schema helps with data quality and import performance. As mentioned during the lesson, we'll create a simple schema to read in the following columns:

Name
Age
City
The `Name` and `City` columns are `StringType()` and the `Age` column is an `IntegerType()`

In [1]:
# Import the pyspark.sql.types library ->Import * from the pyspark.sql.types library.
from pyspark.sql.types import *

# Define a new schema using the StructType method
people_schema = StructType([
  # Define a StructField for each field -> Define a StructField for name, age, and city. Each field should correspond to the correct datatype and not be nullable.
  StructField('name', StringType(), False),
  StructField('age', IntegerType(), False),
  StructField('city', StringType(), False),
])

In [2]:
# to build a new SparkSession
from pyspark.sql import SparkSession
from pyspark import SparkContext

spark= SparkSession.builder.getOrCreate()
    
sc = spark.sparkContext
sc

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/20 11:00:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[*] appName=pyspark-shell>

In [3]:
# check the saved table in spark
spark.catalog.listTables()

[]

### Using lazy processing

- Lazy processing operations will usually return in about the same amount of time regardless of the actual quantity of data. Remember that this is due to Spark not performing any transformations until an action is requested.

- For this exercise, we'll be defining a Data Frame (aa_dfw_df) and add a couple transformations. Note the amount of time required for the transformations to complete when defined vs when the data is actually queried. These differences may be short, but they will be noticeable. When working with a full Spark cluster with larger quantities of data the difference will be more apparen

In [4]:
# Load the CSV file
from pyspark.sql.functions import lower
aa_dfw_df = spark.read.csv('/Users/ssamilozkan/Desktop/pyspark/cleaning_data_with_pyspark/dataset/AA_DFW_2014_Departures_Short.csv', header=True)

# Add the airport column using the F.lower() method
aa_dfw_df = aa_dfw_df.withColumn('airport', lower(aa_dfw_df['Destination Airport']))

# Drop the Destination Airport column
aa_dfw_df = aa_dfw_df.drop(aa_dfw_df['Destination Airport'])

# Show the DataFrame
aa_dfw_df.show(3)


+-----------------+-------------+-----------------------------+-------+
|Date (MM/DD/YYYY)|Flight Number|Actual elapsed time (Minutes)|airport|
+-----------------+-------------+-----------------------------+-------+
|       01/01/2014|         0005|                          519|    hnl|
|       01/01/2014|         0007|                          505|    ogg|
|       01/01/2014|         0035|                          174|    slc|
+-----------------+-------------+-----------------------------+-------+
only showing top 3 rows



### Saving a DataFrame in Parquet format

- When working with Spark, you'll often start with CSV, JSON, or other data sources. This provides a lot of flexibility for the types of data to load, but it is not an optimal format for Spark. The Parquet format is a columnar data store, allowing Spark to use predicate pushdown. This means Spark will only process the data necessary to complete the operations you define versus reading the entire dataset. This gives Spark more flexibility in accessing the data and often drastically improves performance on large datasets.

- In this exercise, we're going to practice creating a new Parquet file and then process some data from it.

- The spark object and the df1 and df2 DataFrames have been setup for you.

In [5]:
df1 = spark.read.csv('/Users/ssamilozkan/Desktop/pyspark/cleaning_data_with_pyspark/dataset/AA_DFW_2014_Departures_Short.csv', header=True)
df2 = spark.read.csv('/Users/ssamilozkan/Desktop/pyspark/cleaning_data_with_pyspark/dataset/AA_DFW_2015_Departures_Short.csv', header=True)
print(type(df1))
print(type(df2))

<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>


In [6]:
# View the row count of df1 and df2
print("df1 Count: %d" % df1.count())
print("df2 Count: %d" % df2.count())

# Combine the DataFrames into one
df3 = df1.union(df2)

# Save the df3 DataFrame in Parquet format
df3.write.parquet('AA_DFW_ALL.parquet', mode='overwrite')

# Read the Parquet file into a new DataFrame and run a count
print(spark.read.parquet('AA_DFW_ALL.parquet').count())

df1 Count: 157198
df2 Count: 146558


303756


### SQL and Parquet

- Parquet files are perfect as a backing data store for SQL queries in Spark. While it is possible to run the same queries directly via Spark's Python functions, sometimes it's easier to run SQL queries alongside the Python options.

- For this example, we're going to read in the Parquet file we created in the last exercise and register it as a SQL table. Once registered, we'll run a quick query against the table (aka, the Parquet file).

- The spark object and the AA_DFW_ALL.parquet file are available for you automatically

In [8]:
# check the saved table in spark
spark.catalog.listTables()

[]

In [9]:
# Read the Parquet file into flights_df
flights_df = spark.read.parquet('AA_DFW_ALL.parquet')
flights_df.show(3)


+-----------------+-------------+-------------------+-----------------------------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|Actual elapsed time (Minutes)|
+-----------------+-------------+-------------------+-----------------------------+
|       01/01/2015|         0005|                HNL|                          526|
|       01/01/2015|         0007|                OGG|                          517|
|       01/01/2015|         0023|                SFO|                          233|
+-----------------+-------------+-------------------+-----------------------------+
only showing top 3 rows



In [10]:
flights_df= flights_df.withColumnRenamed("Actual elapsed time (Minutes)", "flight_duration")
flights_df.show(3)

+-----------------+-------------+-------------------+---------------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|flight_duration|
+-----------------+-------------+-------------------+---------------+
|       01/01/2015|         0005|                HNL|            526|
|       01/01/2015|         0007|                OGG|            517|
|       01/01/2015|         0023|                SFO|            233|
+-----------------+-------------+-------------------+---------------+
only showing top 3 rows



In [11]:
# filter rows where name starts with 'A'
flights_df.filter(flights_df["Destination Airport"].startswith('A')).show(3)

+-----------------+-------------+-------------------+---------------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|flight_duration|
+-----------------+-------------+-------------------+---------------+
|       01/01/2015|         0125|                ATL|            103|
|       01/01/2015|         0194|                ATL|              0|
|       01/01/2015|         0232|                ATL|            113|
+-----------------+-------------+-------------------+---------------+
only showing top 3 rows



In [12]:
# Register the temp table
flights_df.createOrReplaceTempView('flights')

# check the saved table in spark
spark.catalog.listTables()

[Table(name='flights', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [13]:
type(flights_df)
flights_df.show(3)

+-----------------+-------------+-------------------+---------------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|flight_duration|
+-----------------+-------------+-------------------+---------------+
|       01/01/2015|         0005|                HNL|            526|
|       01/01/2015|         0007|                OGG|            517|
|       01/01/2015|         0023|                SFO|            233|
+-----------------+-------------+-------------------+---------------+
only showing top 3 rows



In [14]:
flights_df.printSchema()

root
 |-- Date (MM/DD/YYYY): string (nullable = true)
 |-- Flight Number: string (nullable = true)
 |-- Destination Airport: string (nullable = true)
 |-- flight_duration: string (nullable = true)



In [15]:
# Run a SQL query of the average flight duration
avg_duration = spark.sql('SELECT avg(flight_duration) from flights').collect()[0]
print('The average flight time is: %d' % avg_duration)

The average flight time is: 143


### DataFrame Column Operations


#### Filtering column content with Python
You've looked at using various operations on DataFrame columns - now you can modify a real dataset. The DataFrame `voter_df` contains information regarding the voters on the Dallas City Council from the past few years. This truncated DataFrame contains the date of the vote being cast and the name and position of the voter. Your manager has asked you to clean this data so it can later be integrated into some desired reports. The primary task is to remove any null entries or odd characters and return a specific set of voters where you can validate their information.

This is often one of the first steps in data cleaning - removing anything that is obviously outside the format. For this dataset, make sure to look at the original data and see what looks out of place for the `VOTER_NAME` column.

The `pyspark.sql.functions` library is already imported under the alias F.

In [18]:
voter_df = spark.read.csv('/Users/ssamilozkan/Desktop/pyspark/cleaning_data_with_pyspark/dataset/DallasCouncilVoters.csv', header=True)
voter_df.show(3)

+----------+-------------+-------------------+
|      DATE|        TITLE|         VOTER_NAME|
+----------+-------------+-------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|
|02/08/2017|Councilmember| Philip T. Kingston|
|02/08/2017|        Mayor|Michael S. Rawlings|
+----------+-------------+-------------------+
only showing top 3 rows



In [24]:
# Show the distinct VOTER_NAME entries
distinct_name_voter = voter_df.select("VOTER_NAME").distinct().show(40, truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|VOTER_NAME                                                                                                                                                                                                                                                                                                                                                                                                                 |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [29]:
# Filter voter_df where the VOTER_NAME is 1-20 characters in length
voter_df = voter_df.filter('length(VOTER_NAME) > 0 and length(VOTER_NAME) < 20')
voter_df.show(3)

+----+-----+----------+
|DATE|TITLE|VOTER_NAME|
+----+-----+----------+
+----+-----+----------+



In [30]:

# Filter out voter_df where the VOTER_NAME contains an underscore
voter_df = voter_df.filter(~ F.col('VOTER_NAME').contains('_'))

# Show the distinct VOTER_NAME entries again
voter_df.select('VOTER_NAME').distinct().show(40, truncate=False)

+----------+
|VOTER_NAME|
+----------+
+----------+



### Modifying DataFrame columns

- Previously, you filtered out any rows that didn't conform to something generally resembling a name. Now based on your earlier work, your manager has asked you to create two new columns - `first_name` and last_name. She asks you to split the `VOTER_NAME` column into words on any space character. You'll treat the last word as the `last_name`, and all other words as the `first_name`. You'll be using some new functions in this exercise including `.split()`, `.size()`, and `.getItem()`. The `.getItem(index)` takes an integer value to return the appropriately numbered item in the column. The functions `.split()` and `.size()` are in the `pyspark.sql.functions` library.

- Please note that these operations are always somewhat specific to the use case. Having your data conform to a format often matters more than the specific details of the format. Rarely is a data cleaning task meant just for one person - matching a defined format allows for easier sharing of the data later (ie, Paul doesn't need to worry about names - Mary already cleaned the dataset).

- The filtered voter DataFrame from your previous exercise is available as voter_df. The `pyspark.sql.functions` library is available under the alias F.

In [32]:
# we have to call again csv file to some manipulations
voter_df = spark.read.csv('/Users/ssamilozkan/Desktop/pyspark/cleaning_data_with_pyspark/dataset/DallasCouncilVoters.csv', header=True)
voter_df.show(3)

+----------+-------------+-------------------+
|      DATE|        TITLE|         VOTER_NAME|
+----------+-------------+-------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|
|02/08/2017|Councilmember| Philip T. Kingston|
|02/08/2017|        Mayor|Michael S. Rawlings|
+----------+-------------+-------------------+
only showing top 3 rows



In [34]:
# Add a new column called splits separated on whitespace
voter_df = voter_df.withColumn('split', F.split(voter_df.VOTER_NAME, '\s+'))
voter_df.show(3)

+----------+-------------+-------------------+--------------------+
|      DATE|        TITLE|         VOTER_NAME|               split|
+----------+-------------+-------------------+--------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|[Jennifer, S., Ga...|
|02/08/2017|Councilmember| Philip T. Kingston|[Philip, T., King...|
|02/08/2017|        Mayor|Michael S. Rawlings|[Michael, S., Raw...|
+----------+-------------+-------------------+--------------------+
only showing top 3 rows



In [36]:
# Create a new column called first_name based on the first item in splits
voter_df = voter_df.withColumn('first_name', voter_df.split.getItem(0))
voter_df.show(3)

+----------+-------------+-------------------+--------------------+----------+
|      DATE|        TITLE|         VOTER_NAME|               split|first_name|
+----------+-------------+-------------------+--------------------+----------+
|02/08/2017|Councilmember|  Jennifer S. Gates|[Jennifer, S., Ga...|  Jennifer|
|02/08/2017|Councilmember| Philip T. Kingston|[Philip, T., King...|    Philip|
|02/08/2017|        Mayor|Michael S. Rawlings|[Michael, S., Raw...|   Michael|
+----------+-------------+-------------------+--------------------+----------+
only showing top 3 rows



In [37]:
# Get the last entry of the splits list and create a column called last_name
voter_df = voter_df.withColumn('last_name', voter_df.split.getItem(F.size('split')-1))
voter_df.show(3)

/Users/ssamilozkan/Desktop/pyspark/pvenv/lib/python3.10/site-packages/pyspark/sql/column.py:419: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  warnings.warn(


+----------+-------------+-------------------+--------------------+----------+---------+
|      DATE|        TITLE|         VOTER_NAME|               split|first_name|last_name|
+----------+-------------+-------------------+--------------------+----------+---------+
|02/08/2017|Councilmember|  Jennifer S. Gates|[Jennifer, S., Ga...|  Jennifer|    Gates|
|02/08/2017|Councilmember| Philip T. Kingston|[Philip, T., King...|    Philip| Kingston|
|02/08/2017|        Mayor|Michael S. Rawlings|[Michael, S., Raw...|   Michael| Rawlings|
+----------+-------------+-------------------+--------------------+----------+---------+
only showing top 3 rows



In [39]:
# Drop the splits column
voter_df = voter_df.drop('split')

# Show the voter_df DataFrame
voter_df.show()

+----------+-------------+-------------------+----------+---------+
|      DATE|        TITLE|         VOTER_NAME|first_name|last_name|
+----------+-------------+-------------------+----------+---------+
|02/08/2017|Councilmember|  Jennifer S. Gates|  Jennifer|    Gates|
|02/08/2017|Councilmember| Philip T. Kingston|    Philip| Kingston|
|02/08/2017|        Mayor|Michael S. Rawlings|   Michael| Rawlings|
|02/08/2017|Councilmember|       Adam Medrano|      Adam|  Medrano|
|02/08/2017|Councilmember|       Casey Thomas|     Casey|   Thomas|
|02/08/2017|Councilmember|Carolyn King Arnold|   Carolyn|   Arnold|
|02/08/2017|Councilmember|       Scott Griggs|     Scott|   Griggs|
|02/08/2017|Councilmember|   B. Adam  McGough|        B.|  McGough|
|02/08/2017|Councilmember|       Lee Kleinman|       Lee| Kleinman|
|02/08/2017|Councilmember|      Sandy Greyson|     Sandy|  Greyson|
|02/08/2017|Councilmember|  Jennifer S. Gates|  Jennifer|    Gates|
|02/08/2017|Councilmember| Philip T. Kingston|  

## Conditional DataFrame column operations